In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import KDTree

In [2]:
pd.set_option("display.max_columns", None)

# Read in data

In [3]:
frames = pd.read_csv('../data/cleaned_train.csv')

C:\Users\ryanc\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (47) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
frames.shape

(682154, 60)

In [5]:
frames.head()

,GameId,PlayId,Team,X,Y,S,A,Dis,Orientation,Dir,NflId,DisplayName,JerseyNumber,Season,YardLine,Quarter,GameClock,PossessionTeam,Down,Distance,FieldPosition,HomeScoreBeforePlay,VisitorScoreBeforePlay,NflIdRusher,OffenseFormation,OffensePersonnel,DefendersInTheBox,DefensePersonnel,PlayDirection,TimeHandoff,TimeSnap,Yards,PlayerHeight,PlayerWeight,PlayerBirthDate,PlayerCollegeName,Position,HomeTeamAbbr,VisitorTeamAbbr,Week,Stadium,Location,StadiumType,Turf,GameWeather,Temperature,Humidity,WindSpeed,WindDirection,IsRusher,YardsToTouchdown,YardsToSafety,OffensivePlayer,PlayLeft,X_std,Y_std,Dir_rad,Dir_std,Sx,Sy
0,2017090700,20170907000118,away,73.91,34.84,4.0,1.13,0.40,81.99,177.18,496723,Eric Berry,29,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-0,212,12/29/1988,Tennessee,SS,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8.0,SW,0,65,35,0,1,46.09,18.46,4.761607,1.620015,-0.196794,3.995156
1,2017090700,20170907000118,away,74.67,32.64,0.1,1.35,0.01,27.61,198.70,2495116,Allen Bailey,97,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-3,288,03/25/1989,Miami,DE,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8.0,SW,0,65,35,0,1,45.33,20.66,4.386012,1.244420,0.032061,0.094721
2,2017090700,20170907000118,away,74.00,33.20,3.1,0.59,0.31,3.01,202.73,2495493,Justin Houston,50,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-3,270,01/21/1989,Georgia,DE,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8.0,SW,0,65,35,0,1,46.00,20.10,4.315676,1.174083,1.197806,2.859241
3,2017090700,20170907000118,away,71.46,27.70,0.2,0.54,0.02,359.77,105.64,2506353,Derrick Johnson,56,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-3,245,11/22/1982,Texas,ILB,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8.0,SW,0,65,35,0,1,48.54,25.60,6.010216,2.868623,-0.192595,0.053918
4,2017090700,20170907000118,away,69.32,35.42,1.6,2.43,0.16,12.63,164.31,2530794,Ron Parker,38,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-0,206,08/17/1987,Newberry,FS,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8.0,SW,0,65,35,0,1,50.68,17.88,4.986231,1.844638,-0.432692,1.540382


# Extract KNN Yard Features

I will build a KD-Tree on rusher features and identify most similar other rushing attempts based on features. I will then extract yard information from these most similar attempts as new features.

In [6]:
rb = frames[frames['IsRusher'] == 1][['GameId','PlayId','X_std','Y_std','Sx','Sy','Dir_std','YardsToTouchdown','Yards']].reset_index(drop=True)

In [7]:
tree = KDTree(rb.iloc[:, 2:-1])

In [8]:
dist, ind = tree.query(rb.iloc[:, 2:-1], k=10)

In [9]:
avgs = []
stds = []
for indx, row in rb.iterrows():
    similar_records = rb.iloc[ind[indx][1:], :]
    
    avg_yards = similar_records.Yards.mean()
    std_yards = similar_records.Yards.std()
    
    avgs.append(avg_yards)
    stds.append(std_yards)

In [10]:
rb['avg_similar_yards'] = avgs
rb['std_similar_yards'] = stds

In [11]:
rb.head()

,GameId,PlayId,X_std,Y_std,Sx,Sy,Dir_std,YardsToTouchdown,Yards,avg_similar_yards,std_similar_yards
0,2017090700,20170907000118,41.25,22.77,3.464423,1.561336,0.423417,65,8,3.111111,1.452966
1,2017090700,20170907000139,48.93,26.14,2.518736,-2.283850,5.546656,57,3,4.000000,3.082207
2,2017090700,20170907000189,71.34,34.19,4.011670,4.461671,0.838456,35,5,2.777778,3.597839
3,2017090700,20170907000345,104.47,27.94,4.579282,-0.436095,6.188239,2,2,1.000000,0.707107
4,2017090700,20170907000395,29.99,27.12,1.653964,-4.077304,5.097758,75,7,3.555556,4.719934


In [12]:
rb.drop('Yards', axis=1, inplace=True)

# RB vs Def Distances and Speed Differences

In [25]:
d = frames[frames['OffensivePlayer'] == 0][['GameId','PlayId','X_std','Y_std','Sx','Sy']]

o = rb[['GameId','PlayId','X_std','Y_std','Sx','Sy']]
o.columns = ['GameId','PlayId','rbX','rbY','rbSx','rbSy']

rb_d = pd.merge(o, d, on=['GameId','PlayId'], how='inner')
rb_d['dist'] = np.sqrt((rb_d['rbX'] - rb_d['X_std'])**2 + (rb_d['rbY'] - rb_d['Y_std'])**2)
rb_d['Sx_diff'] = rb_d['rbSx'] - rb_d['Sx']
rb_d['Sy_diff'] = rb_d['rbSy'] - rb_d['Sy']

In [26]:
rb_d.head()

,GameId,PlayId,rbX,rbY,rbSx,rbSy,X_std,Y_std,Sx,Sy,dist,Sx_diff,Sy_diff
0,2017090700,20170907000118,41.25,22.77,3.464423,1.561336,46.09,18.46,-0.196794,3.995156,6.480872,3.661217,-2.433820
1,2017090700,20170907000118,41.25,22.77,3.464423,1.561336,45.33,20.66,0.032061,0.094721,4.593310,3.432362,1.466615
2,2017090700,20170907000118,41.25,22.77,3.464423,1.561336,46.00,20.10,1.197806,2.859241,5.448982,2.266617,-1.297905
3,2017090700,20170907000118,41.25,22.77,3.464423,1.561336,48.54,25.60,-0.192595,0.053918,7.820038,3.657018,1.507418
4,2017090700,20170907000118,41.25,22.77,3.464423,1.561336,50.68,17.88,-0.432692,1.540382,10.622476,3.897115,0.020954


In [43]:
plays = list(set(rb_d.PlayId))
rb_v_def = np.zeros((rb.shape[0], 34))
for indx,play in enumerate(plays):
    X = rb_d[rb_d['PlayId'] == play]
    dists = sorted(abs(X.dist))
    Sxs = sorted(abs(X.Sx_diff))
    Sys = sorted(abs(X.Sy_diff))
    
    concat = [play] + dists + Sxs + Sys
    rb_v_def[indx] = concat

In [45]:
rb_v_def_cols = ['PlayId']+['dist_{}'.format(x) for x in range(1,12)]+['Sx_{}'.format(x) for x in range(1,12)]+['Sy_{}'.format(x) for x in range(1,12)]

In [46]:
rb_v_def = pd.DataFrame(rb_v_def, columns=rb_v_def_cols)

In [47]:
rb_v_def.head()

,PlayId,dist_1,dist_2,dist_3,dist_4,dist_5,dist_6,dist_7,dist_8,dist_9,dist_10,dist_11,Sx_1,Sx_2,Sx_3,Sx_4,Sx_5,Sx_6,Sx_7,Sx_8,Sx_9,Sx_10,Sx_11,Sy_1,Sy_2,Sy_3,Sy_4,Sy_5,Sy_6,Sy_7,Sy_8,Sy_9,Sy_10,Sy_11
0,2.017122e+13,4.980371,6.123341,6.133653,6.305371,7.568619,7.929552,8.203231,9.030138,15.164333,19.860337,20.148193,1.645130,3.933083,3.939058,4.273578,4.746555,4.800291,5.118860,5.149457,5.824494,6.822195,7.524569,0.058056,0.082171,0.175214,0.199471,0.633709,1.032713,1.033086,1.250960,1.310765,1.552749,4.563727
1,2.017102e+13,3.611385,4.186454,5.174292,5.628934,5.938560,6.037922,6.492496,8.081844,15.028187,21.420936,22.406751,0.096411,0.970886,3.946302,4.105743,4.206641,4.407447,4.468128,4.862503,4.956113,5.267628,5.959236,0.374864,0.420469,0.507231,0.595721,0.683713,1.199014,1.397915,1.545032,1.583429,1.957199,4.091698
2,2.019111e+13,6.223825,6.282579,6.338178,6.791657,8.129367,9.612934,9.647974,10.057003,13.993088,17.977033,25.961096,1.047170,1.679977,2.023219,2.331405,2.333182,2.387806,2.692220,2.823091,3.148800,4.160385,6.369028,0.397068,0.440723,0.534730,0.551776,1.194663,1.802009,2.050078,2.312258,3.176280,3.332715,4.283589
3,2.019101e+13,2.080601,4.023494,4.231040,6.189426,12.585027,12.999342,16.770107,18.730331,22.117588,25.638645,26.570736,0.049934,0.111979,0.172501,0.647995,0.688557,1.389108,1.804632,3.335155,4.103780,4.221909,5.398674,0.174722,0.213715,0.305715,0.358884,0.572271,1.057914,1.126619,2.076124,2.582258,2.926829,4.101273
4,2.018101e+13,5.573554,5.662102,5.873781,7.898158,8.165341,9.860832,9.892568,14.557094,14.825148,22.138349,23.192973,0.997956,1.943330,2.096194,2.200059,2.281270,2.608214,3.037683,3.063134,3.705738,3.729659,3.854498,0.078232,0.235644,0.356157,0.517638,0.541564,0.667097,0.703531,1.091624,1.151643,1.719441,1.720818


In [48]:
rb_features = pd.merge(rb, rb_v_def, on='PlayId', how='inner')

In [49]:
rb_features.head()

,GameId,PlayId,X_std,Y_std,Sx,Sy,Dir_std,YardsToTouchdown,avg_similar_yards,std_similar_yards,LOS,dist_1,dist_2,dist_3,dist_4,dist_5,dist_6,dist_7,dist_8,dist_9,dist_10,dist_11,Sx_1,Sx_2,Sx_3,Sx_4,Sx_5,Sx_6,Sx_7,Sx_8,Sx_9,Sx_10,Sx_11,Sy_1,Sy_2,Sy_3,Sy_4,Sy_5,Sy_6,Sy_7,Sy_8,Sy_9,Sy_10,Sy_11
0,2017090700,20170907000118,41.25,22.77,3.464423,1.561336,0.423417,65,3.111111,1.452966,45,4.593310,4.880256,5.448982,6.480872,7.500467,7.820038,9.902020,10.622476,12.968593,14.644511,22.415872,2.164424,2.266617,2.505281,3.342920,3.364684,3.432362,3.657018,3.661217,3.897115,5.257546,5.766705,0.020954,1.297905,1.404143,1.466615,1.507418,1.562244,1.568556,1.720198,2.433820,3.154917,3.447660
1,2017090700,20170907000139,48.93,26.14,2.518736,-2.283850,5.546656,57,4.000000,3.082207,53,4.287773,5.322875,5.821151,6.658686,7.760058,8.583356,10.224094,10.579986,12.301711,18.701749,23.025872,0.137323,0.350645,1.710355,1.990288,2.422243,2.656235,2.894865,2.972321,3.012734,3.073480,4.087800,0.057249,0.388832,0.490772,1.108155,1.624866,2.310103,3.301946,3.457160,4.221881,4.357021,4.406432
2,2017090700,20170907000189,71.34,34.19,4.011670,4.461671,0.838456,35,2.777778,3.597839,75,4.221670,4.982550,6.074084,6.690030,8.175292,8.387163,8.406343,11.708224,14.379447,15.189486,20.726285,1.601986,1.931711,2.857983,3.185681,3.517688,3.560234,3.700775,4.230076,4.432300,4.914459,5.072099,0.088401,0.155283,0.391538,0.467638,0.471210,0.774755,1.091774,1.163528,1.491305,1.923281,4.791832
3,2017090700,20170907000345,104.47,27.94,4.579282,-0.436095,6.188239,2,1.000000,0.707107,108,4.528002,4.596618,4.688123,4.895651,5.704849,5.770442,5.771464,6.864197,7.754953,9.037367,9.791231,3.653618,3.897596,3.975188,4.101873,4.366510,4.400146,4.946050,5.833505,5.922531,5.947311,7.222570,0.715022,0.778036,0.846374,0.900058,1.286837,1.336797,1.359690,1.428169,1.455441,1.672383,2.728092
4,2017090700,20170907000395,29.99,27.12,1.653964,-4.077304,5.097758,75,3.555556,4.719934,35,4.288088,5.791312,5.902169,6.538333,7.188992,9.476086,10.557391,15.539743,16.575367,18.548730,21.214806,0.150767,1.299483,1.482852,1.653964,2.114991,2.349646,2.417051,2.804814,2.850957,3.094264,3.645783,0.851466,0.866743,0.887392,0.896998,0.959186,1.112940,2.206368,2.351845,3.502220,4.077304,5.589105


In [50]:
rb_features.to_csv('../data/rusher_features.csv', header=True, index=False)